In [ ]:
Assistants ajudam a criar assistentes especificos sobre um determinado tipo de assunto.
Respondem assuntos complexos e especificos(com base num arquivo .csv ou .pdf por exemplo)
Esse código é o basico para a criaçao do modelo, alimentando com as informações e obtendo o resultado
Esse notebook é um exemplo completo de como usar a API da OpenAI para:
Criar um assistente especializado.
Enviar perguntas técnicas.
Aguardar e recuperar respostas.
Auditar os passos internos da execução.
46. Explorando Assistants

import openai
import time


In [ ]:
client = openai.Client()

Cria um Assistant personalizado com:

Nome: "Tutor de Tecnologia"
Instruções: foco em tecnologia
Ferramenta: code_interpreter
Modelo: gpt-3.5-turbo-0125

In [ ]:
assistant = client.beta.assistants.create(
    name="Tutor de Tecnologia",
    instructions="Você é um tutor sobre assuntos relacionados a tecnologia",
    tools=[{"type":"code_interpreter"}],
    model="gpt-3.5-turbo-0125")

In [ ]:

pergunta = """
Em um sistema de cache com mapeamento direto, dado uma memória principal de uma cache
de 256 KB e blocos de 64 bytes:
- Quantas linhas existem na cache?
- Onde o bloco de endereço 0x1A23C será mapeado na cache?
"""

In [ ]:

# Criação da Thread

thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=pergunta)


In [ ]:

# Executa a thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Nome de usuário premium"
)


In [ ]:
# mostra o estagio da solicitação que voce enviou. Pode ser "executado", ou estar no estagio 
# de "fila/queued" por exemplo
run.status

: 

In [ ]:
# Aguarda a thread rodar
while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )


In [ ]:

# Verifica a resposta quando tivermos a resposta da thread
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens.data[0].content[0].text.value)
else:
    print(f"Erro {run.status}")


Verifica se a execução foi concluída e tenta imprimir a resposta. 

In [ ]:
print(mensagens.data[0].content[0].text.value)

In [ ]:
#analisando a logica/passos do modelo para chegar na resposta
run_steps = client.beta.threads.run.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

Imprime os detalhes de cada passo:
Se o modelo usou o code_interpreter, mostra o código executado.
Se foi uma criação de mensagem, mostra o conteúdo gerado.

In [ ]:

for step in run_steps.data[::-1]:
    print(f"\n===Step {step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("-" * 10)
            print(tool_call.code_interpreter.input)
            print("-" * 10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
        thread_id=thread.id,
        message_id=step.step_details.message_creation.message_id
    )

print(message.content[0].text.value)